# Project Details
> What is the idea behind our Project?
- toc: true
- title: Project Details
- author: Alex Lu
- badges: true
- comments: true
- categories: [jupyter,APCSP,week 5]
- author: Alex Lu

## The Idea

Our team wishes to build a minesweeper game with a python backend that could consistently communicate with our front-end website to bring a new degree of sophistication and aesthetic to this classic Game. This project would be split into two main portions each with it's own challenge. The first portion would be making the game engine in python, while the latter portion would be to design and create an aesthetically pleasing and functional frontend to display the Game Status

In [2]:
#hide_input
from IPython.display import Image

# get the image
Image(url="https://ylu-1258.github.io/YLu-Blog/images/minesweeper_diagram.png")

## The Frontend
The frontend aspect of this project should preferably display a grid of the game and incorporate some aspect of user input. Preliminary testing and experimentation could be done through textbox inputs of coordinate values, while prospective functions could include individual buttons on the Grid itself to provide a GUI for the user to interact with. In both cases, the frontend should be able to communicate with the python backend through JSON data to send and receive data.  

More aspects about the Frontend aspects will be added with additional planning in the future.

![]({{site.baseurl}}/images/minesweeper_diagram.png "Minesweeper design methodology and interactions")

## The Backend
Arguably the more challenging half of this project, the backend would primarily serve to be the game engine of the minesweeper game. Additionally, the backend would also control and organize the various pages and menus on the website. In this writeup, we will only be focusing on the main goals and challenges faced by the Backend team.

1. Using Object Oriented Programming to create individual objects for the game. Some aspects that could be represented by such objects would be the gameboard and also the individual cells and nodes present within the game board.
    1. Doing so would simplify the algorithmic aspect of the game, as the board itself could have methods and attributes to help control the game logic. 
    2. Morever, using an object to represent an individual cell in the board would provide greater functionalities than just using a single variable  
2. Use Enumerated types with the python `enum` module to create different values for the type and status of each node. Each of these types could then be bound to a constant value which could then be printed on the screen

3. Use Recursion to create an algorithm to recursively detect adjacent cells that are safe (AKA not mines). This algorithm would work in the following format
    1. Maintain a list of current cells already determined to be safe
    2. Verify if the four adjacent (up, down, left, right) cells next to the selected cell are safe or not, if safe, store the coordinate point in the list, if not, record the cell as a "border cell", terminate the recursive process, and run another helper function to determine the precise number of mines surrounding the cell
    3. Re-call the function for each of the surrounding adjacent cells to identify other consecutive cells who are safe.
    4. Return a list of the coordinates of a contiguous block of safe cells
    5. Mark the cells to be safe and calculate number of surrounding mines for border cells.

4. Verify the Game status, Game is won if:
    1. All mines are flagged
    2. All safe cells are cleared
    Game is lost if:
    1. A mine was dug by the user

5. Return the final result back to the front end. If a safe cell was dug, send out a JSON containing an array of the coordinates of the cell and it's safe neighbors. If a mine was dug, send a JSON containing a boolean value to signify the end of the game. 


## **CHALLENGE 1:** OOP
Below is the currently implemented code for the gameboard and each individual node.  

Here is the code for each individual node in the minesweeper:

In [3]:
from enum import Enum
from random import randint

In [4]:
class MType(Enum):
    SAFE = 1
    MINE = 2     

class MStatus(Enum):
    UNKNOWN = 1
    DUG = 2  
    FLAG = 3  

class Node:
    def __init__(self):
        self.type = MType.SAFE
        self.status = MStatus.UNKNOWN
        self.value = "-"
    
    def setInitialValue(self):
        if self.type == MType.MINE:
            self.value="M"

Here is the Code for the Minesweeper gameboard and its methods.

In [5]:
# Custome Errors
class NotInBoard(Exception):
    pass

class Minesweeper:
    # Class constructor, containing game data
    def __init__(self):
        self.mine_coords = []
        self.board = []
        self.rows = 0
        self.cols = 0
        self.gameWin = False
        self.gameOver = False

    # Create the gameboard on a given input of rows and columns
    def generateBoard(self,r,c):
        self.rows = r
        self.cols = c
        self.board = [[Node() for i in range(self.rows)] for i in range(self.cols)]
        return self.board

    # Print the current board
    def printBoard(self):
        res = ""
        for row in self.board:
            concat = ""
            for column in row:
                concat = concat + column.value + " "
            concat = concat.strip() + "\n"
            res = res+concat
        print(res)
        return 0

    # Generate the coordinates of the mines AFTER the first user input
    # THE FIRST ACTION ON A CELL WOULD NEVER BE A MINE!
    def setMines(self):
        num_of_mines = (self.rows * self.cols)//4
        counter = 0
        while counter <= num_of_mines:
            mine_coord = (randint(0,self.rows-1),randint(0,self.cols-1))
            print("Mine coord: ", mine_coord)
            if mine_coord in self.mine_coords:
                continue
            self.board[mine_coord[0]][mine_coord[1]].type = MType.MINE
            self.mine_coords.append(mine_coord)
            counter +=1
        for i in range(self.rows):
            for j in range(self.cols):
                self.board[i][j].setInitialValue()
        return 0

    # The first click of the game
    def firstClick(self):
        self.printBoard()
        try:
            ipt = input("What row and column? (format in row,column): ")
            row, col = int(ipt.split(",")[0]), int(ipt.split(",")[1])
            if row not in range(1,self.rows+1) or col not in range(1,self.cols+1):
                print("Values not in bound, try again")
                raise NotInBoard
            self.setMines()
        except NotInBoard:
            self.firstClick()

    # Following actions after the first-dig
    # checkAdkacent function will be created soon
    def digMine(self, r ,c):
        if self.board[r][c].type == MType.MINE:
            self.gameOver = True
        else:
            safe_mines = self.checkAdjacent(r,c)

## **CHALLENGE 2:** Enumerated types
This part is kinda free tho ngl

In [6]:
from enum import Enum

class MType(Enum):
    SAFE = 1
    MINE = 2     

class MStatus(Enum):
    UNKNOWN = 1
    DUG = 2  
    FLAG = 3  

## **CHALLENGE 3:** Recursive algorithm

In [ ]:
def revealCells(self, r, c):
    if r < 0 or r >= self.rows or c < 0 or c >= self.cols:
        return
    if self.board[r][c].type == MType.MINE:
        return
    elif self.board[r][c].adjacent == 0 and self.board[r][c].display == "-":
        self.safe_field.append([(r,c)])
        self.board[r][c].reveal()
        self.revealCells(r-1,c-1)
        self.revealCells(r-1,c)
        self.revealCells(r-1,r+1)
        self.revealCells(r,c-1)
        self.revealCells(r,c+1)
        self.revealCells(r+1,c-1)
        self.revealCells(r+1,c)
        self.revealCells(r+1,c+1)

## **CHALLENGE 4:** Game Status

Yet to be implemented

## **CHALLENGE 5:** JSON communication

Yet to be implemented

## **Current Game Implementation**

In [8]:
def main():
    MS = Minesweeper()
    rows = int(input("How many rows do you want? "))
    cols = int(input("How many columns do you want? "))

    MS.generateBoard(cols, rows)
    MS.firstClick()
    MS.printBoard()
    # Define a while lop here to loop game.
    print("List of generated mine coordinates: ", MS.mine_coords)

main()

- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -

Mine coord:  (3, 8)
Mine coord:  (9, 2)
Mine coord:  (4, 7)
Mine coord:  (7, 3)
Mine coord:  (9, 6)
Mine coord:  (8, 1)
Mine coord:  (5, 1)
Mine coord:  (7, 6)
Mine coord:  (5, 5)
Mine coord:  (8, 4)
Mine coord:  (7, 2)
Mine coord:  (4, 7)
Mine coord:  (8, 3)
Mine coord:  (2, 9)
Mine coord:  (0, 9)
Mine coord:  (5, 4)
Mine coord:  (7, 9)
Mine coord:  (2, 0)
Mine coord:  (0, 1)
Mine coord:  (4, 4)
Mine coord:  (4, 2)
Mine coord:  (6, 3)
Mine coord:  (0, 6)
Mine coord:  (3, 0)
Mine coord:  (9, 7)
Mine coord:  (8, 3)
Mine coord:  (4, 9)
Mine coord:  (9, 6)
Mine coord:  (3, 9)
- M - - - - M - - M
- - - - - - - - - -
M - - - - - - - - M
M - - - - - - - M M
- - M - M - - M - M
- M - - M M - - - -
- - - M - - - - - -
- - M M - - M - - M
- M - M M - - - - -
- - M - - - M M - -

List of generated 

<img src="https://ylu-1258.github.io/YLu-Blog/images/scrum_methodology.png" alt="Scrum Process">

In [7]:
#hide_input
from IPython.display import Image

# get the image
Image(url="https://ylu-1258.github.io/YLu-Blog/images/scrum_methodology.png")

## Collegeboard's 6 main criteria

### Program Purpose & Function

- An online single-player minesweeper game that can be played anywhere, anytime  
- Users could choose to store data such as high scores (minimum number of moves or time) if they wish.

### Data Abstraction
- Using the `sqlite3` library on python to store user names and each user's corresponding information, such as high-score
- Using python lists to store the state of the game, what nodes or cells are revealed, flagged, hidden, etc.
- Creating a main class for the game and then establishing a constructor that initializes other object attributes for the main game to access or manipulate.

### Managing Complexity
- Maintaining user data in a organized manner with software like databases (SQL).
- Using loops to reduce repetitive code / allow for a dynamic number of times for the execution of our code.

### Procedural Abstraction
- Establishing a set of enumerated types to link certain values with constants. 
    - Used to established the different states of a node or the particular status of that cell.
- Using a main class to cover the Minesweeper Game:
    - The class will contain a constructor, attributes, and methods to ensure smooth gameplay.
    - This will simplify the programming process too as each individual method under the class will "inherit" the attributes of the class. This expands the scope of our functions sort of "links" them together. (Methods work in harmony with each other)
- Creating templates for SQL queries, JSON queries, and also html pages to reduce the amount of manual programing required

### Algorithm Implementation
- Maintain a clear-board at the begining of the game. On the first input (click) from the user, start randomly generating `N=(M//3)` number of mines, where M is the total number of mines. 
    - Use the `random` module to generate a set of integers within the bounds of the array, generate two integers from each mine to indicate the row or coloumn coordinate. Finally, add both generated values into a tuple which will then be appended to a list containing all of the mine coordinates.
    - AS OF **10/10** Consider using the `RapidAPI minesweeper API` to generate a board. (However this limits our flexibility as the first mine clicked might be an actual mine, leading to an inevitable loss).
- Use an overall gameloop that checks the status of the game on each "turn of the game" (iteration of the game).
    - If `gameOver == true`, signify that the game is over
        - Use control flow statements such as if-else-elif to determine wether or not a user has clicked on a mine. If yes, set `gameOver` to `true`, if not, continue with the current iteration of the loop. 
        - Based on the current state of the board, if there are no mines left, return a win screen. If there are mines left, assume that the player clicked on a mine and return a lose screen.
- Within each iteration, prompt the user to input a coordinate point (When testing, use terminal input with standard `(x,y)` input, for the final product, clicking each cell should return a coordinate automatically to the python backend).
    - **Hard part is detecting a nearby "safe grid" for each mine clicked, while subsequently revealing all the mines in the safe grid**
    - We have decided to use a **recursive** approach to resolve this problem. The recursive approach will store a list of safe mines. It will first append the mine clicked by the user (assuming that the mine passes the safe check). Following this, the algorithm will then check the `(r+1,c), (r+1,c-1), (r+1,c+1), (r,c-1), (r,c+1), (r-1,c), (r-1,c-1), and (r-1,c+1)` coordinates to determine wether or not surrouding cells are safe or not. Recursively execute the same process for each the 8 mines adjacent to the initial cell and stop until an "edge-mine" is reached.
- Communiate user input and program output via JSON queries between the python backend and the HTML/CSS/JS frontend.